In [23]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
import os

# 데이터 폴더 경로 설정
data_folder = 'ml-latest-small'

# 데이터 로드
movies = pd.read_csv(os.path.join(data_folder, 'movies.csv'))
ratings = pd.read_csv(os.path.join(data_folder, 'ratings.csv'))

# Surprise 데이터셋 생성
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# 훈련 세트와 테스트 세트로 분할
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# SVD 모델 (잠재 요인 협업 필터링)
svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
svd.fit(trainset)

# KNN 모델 (아이템 기반 협업 필터링)
knn = KNNBasic(sim_options={'name': 'cosine', 'user_based': False})
knn.fit(trainset)

# 하이브리드 모델 (SVD와 KNN의 예측을 결합)
def hybrid_predict(uid, iid, r_ui=None):
    svd_pred = svd.predict(uid, iid, r_ui)
    knn_pred = knn.predict(uid, iid, r_ui)
    return (svd_pred.est + knn_pred.est) / 2

# 모델 평가
svd_accuracy = accuracy.rmse(svd.test(testset))
knn_accuracy = accuracy.rmse(knn.test(testset))

# 하이브리드 모델 예측 및 정확도 계산
hybrid_predictions = []
for uid, iid, true_r in testset:
    pred = hybrid_predict(uid, iid)
    hybrid_predictions.append((uid, iid, true_r, pred, None))

hybrid_accuracy = accuracy.rmse(hybrid_predictions)

print(f"SVD RMSE: {svd_accuracy}")
print(f"KNN RMSE: {knn_accuracy}")
print(f"Hybrid RMSE: {hybrid_accuracy}")

# 모델 저장
import joblib
joblib.dump((svd, knn), 'hybrid_model.joblib')

print("모델 훈련 및 저장 완료")

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8832
RMSE: 0.9826
RMSE: 0.9078
SVD RMSE: 0.8832365645948294
KNN RMSE: 0.9826409877158173
Hybrid RMSE: 0.9077694925505662
모델 훈련 및 저장 완료
평점 매긴 영화수: 232 추천대상 영화수: 9510 전체 영화수: 9742
##### 사용자 1를 위한 Top-10 추천 영화 리스트 #####
Jumanji (1995) : 3.50
Waiting to Exhale (1995) : 3.50
Father of the Bride Part II (1995) : 3.50
Sabrina (1995) : 3.50
Tom and Huck (1995) : 3.50
Sudden Death (1995) : 3.50
GoldenEye (1995) : 3.50
American President, The (1995) : 3.50
Dracula: Dead and Loving It (1995) : 3.50
Balto (1995) : 3.50
